In [1]:
import torch
import numpy as np
from datasets import load_dataset
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

/home/arjun/Desktop/AI_ENV/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
batch_size = 256

# Creating vocabulory

In [3]:
n = 3
embedding_dims = 10 # how many dimensional vector should represent each word in the vocabulory

In [4]:
ds = load_dataset("MuskumPillerum/General-Knowledge")
df = ds['train'][:10]

In [5]:
def tokenize(sentence):
    data = sentence
    split_terms = [',', '.', '!', '?', '(', ')', '&', '$', '+', '-', '/', '*', ';', ':']
    for split_term in split_terms:
        if split_term in sentence:
            data = data.replace(split_term, f' {split_term} ')
    data = data.split()
    return data

tokenize('what is your !! ! name ?    ')

['what', 'is', 'your', '!', '!', '!', 'name', '?']

In [6]:
vocab_list = set()
X = []
for x,y in zip(df['Question'], df['Answer']):
    data = f'Question: {x} Answer: {y}'
    data = data.lower().replace('\\n', '')
    vocab_list.update(tokenize(data))
    X.append(data)

In [7]:
vocab_list.add('<UNK>')

In [8]:
vocab = {v:i for v,i in zip(vocab_list, range(0, len(vocab_list)+1))}
len(vocab)

141

In [9]:
vocab

{'of': 0,
 'algorithm': 1,
 'maximize': 2,
 'machine': 3,
 'while': 4,
 'can': 5,
 'making': 6,
 'predictions': 7,
 'natural': 8,
 'to': 9,
 'feedback': 10,
 'systems': 11,
 'algorithms': 12,
 'type': 13,
 'artificial': 14,
 'such': 15,
 'vision': 16,
 'penalty': 17,
 'intelligence': 18,
 'relationships': 19,
 'around': 20,
 'general': 21,
 'as': 22,
 'task': 23,
 'over': 24,
 'main': 25,
 'refers': 26,
 'processing': 27,
 'given': 28,
 'perform': 29,
 'understand': 30,
 'time': 31,
 'what': 32,
 'neural': 33,
 'without': 34,
 'answer': 35,
 'aims': 36,
 'environment': 37,
 '-': 38,
 'have': 39,
 'being': 40,
 'way': 41,
 'analyze': 42,
 'many': 43,
 'and': 44,
 'complex': 45,
 'focuses': 46,
 'on': 47,
 'by': 48,
 'consists': 49,
 'improve': 50,
 'inspired': 51,
 'categories': 52,
 'us': 53,
 'the': 54,
 'development': 55,
 'ability': 56,
 'require': 57,
 'make': 58,
 'decision': 59,
 'interpret': 60,
 'computer': 61,
 '.': 62,
 'visual': 63,
 'transmit': 64,
 'would': 65,
 'recogniti

In [10]:
embedding_matrix = torch.randn(len(vocab),embedding_dims)
embedding_matrix.shape

torch.Size([141, 10])

In [11]:
embedding_matrix[[1,2,3]]

tensor([[-0.9709, -0.6108, -0.7059,  0.6737, -0.0704, -0.4620, -0.1340, -2.8420,
          1.2939, -1.2586],
        [ 2.0123,  1.1058, -1.2964, -0.3664,  1.3379,  0.7395, -0.2012,  0.5657,
          0.9096,  1.8883],
        [ 1.1060,  2.8013,  0.6866, -0.2444,  0.1662, -1.4238,  0.2097,  0.8396,
         -1.5886,  1.3464]])

In [12]:
def get_word_embedding(word, vocab=vocab, embedding_matrix=embedding_matrix):
    if word not in vocab:
        word = '<UNK>'
    embedding = embedding_matrix[vocab[word]]
    return embedding

get_word_embedding('as')

tensor([ 0.5536, -1.4848, -1.3051,  0.0212, -1.0637,  0.3486, -0.3957,  0.6814,
         0.3893,  2.1993])

# Creating the model

In [13]:
from print_color import print

In [14]:
class NeuralNetwork(nn.Module):
    def __init__(self, n=n, num_hidden_layer=1024, vocab_len = len(vocab_list), dim_embedding=10):
        super(NeuralNetwork, self).__init__()
        self.n = n
        self.dim_embedding = dim_embedding
        # self.embedding = torch.rand(vocab_len, dim_embedding).to(device)
        self.embedding = nn.Embedding(vocab_len, dim_embedding).to(device)

        self.hidden_layer = nn.Linear((n-1)*dim_embedding, num_hidden_layer)
        self.relu = nn.ReLU()
        self.output = nn.Linear(num_hidden_layer, vocab_len)
        # self.softmax = nn.Softmax(dim=1)

    def forward(self,x):
        # x will be the indeices of embedding representing the input words
        # print(self.embedding(x), color='purple')
        x_embeddings = self.embedding(x).view(-1,(self.n-1)*self.dim_embedding).to(device)

        out = self.hidden_layer(x_embeddings)
        out = self.relu(out)
        out = self.output(out)
        # out = self.softmax(out)
        return out

model = NeuralNetwork(n=3).to(device)

# Creating dataset

In [15]:
dataset = []
for x,y in zip(df['Question'], df['Answer']):
    data = f'Question: {x} Answer: {y}'.lower()
    tokenized_data = tokenize(data)
    for i in range(len(tokenized_data)-n):
        # print(i)
        data_i = tokenized_data[i:i+n]
        dataset.append([vocab[i] if i in vocab else vocab['<UNK>'] for i in data_i])
    # print()

dataset_np = np.array(dataset)
dataset_np.shape

(419, 3)

In [16]:
class QuestionAnswerDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.x = dataset[:,[i for i in range(n-1)]]
        self.y = dataset[:,-1]
        self.m, self.n = self.x.shape
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.m
    
dataset = QuestionAnswerDataset(dataset=dataset_np)
dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

# Training

In [17]:
num_epochs = 100
lr = 0.01
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)

cuda


In [18]:
model = torch.compile(model)

In [19]:
lossCategory = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(), lr=lr)

In [20]:
for epoch in range(num_epochs):
    for i, (input, label) in enumerate(dataloader):
        x = input.to(device)
        y = label.to(device)
        y_pred = model(x)
        loss = lossCategory(y_pred, y)
        print(loss.item())
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()


/home/arjun/Desktop/AI_ENV/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:150: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


4.949789047241211
4.9633331298828125
4.935501575469971
4.928872108459473
4.910515308380127
4.910934925079346
4.892205238342285
4.883504390716553
4.864453315734863
4.872479438781738
4.85221004486084
4.837987899780273
4.8251423835754395
4.825071334838867
4.798806667327881
4.813121795654297
4.786582946777344
4.778018951416016
4.785912036895752
4.7265753746032715
4.746213436126709
4.734324932098389
4.702948570251465
4.750696182250977
4.71498441696167
4.680456161499023
4.711821556091309
4.635204315185547
4.6762919425964355
4.639272212982178
4.638580322265625
4.648630619049072
4.6040120124816895
4.653141021728516
4.639750957489014
4.549374103546143
4.602304458618164
4.557648181915283
4.560262680053711
4.577327251434326
4.585618019104004
4.489538192749023
4.527306079864502
4.534140110015869
4.5193400382995605
4.500542163848877
4.480415344238281
4.516519546508789
4.461699485778809
4.501625061035156
4.437311172485352
4.496888637542725
4.429672718048096
4.466240882873535
4.421779632568359
4.4351